In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(5603, 12)

In [3]:
len(df[df.tiling_compatible == True]) 

181

In [4]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   2052
1  cant_extract_variables   1949
2        no_granule_found    971
3  tile_generation_failed    402
4                    None    181
5                 timeout     48


In [49]:
import numpy as np

# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and x.size > 0)
)]
subset_df

,collection_concept_id,concept_id,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
2,C2069247178-GES_DISC,G2071455506-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
4,C2033151148-GES_DISC,G2033667526-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, Tair, Qair, PSurf, Wind_E, Wind_N,..."
6,C2069247359-GES_DISC,G2076482666-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
7,C2033167496-GES_DISC,G2035054839-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, Rainf, CRainf, ACond, Tair..."
10,C2021957295-LPCLOUD,G2705887278-LPCLOUD,s3://lp-prod-protected/HLSS30.020/HLS.S30.T33K...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[NIR_Narrow]
...,...,...,...,...,...,...,...,...,...,...,...,...
2910,C1652491691-GES_DISC,G1652768767-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
2912,C1652491711-GES_DISC,G1652768756-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
2914,C1652491837-GES_DISC,G1652768721-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
3222,C3271567778-NSIDC_CPRD,G3271605286-NSIDC_CPRD,s3://nsidc-cumulus-prod-protected/SNOWEX/SNEX2...,rasterio,None,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]


In [61]:
subset_df[subset_df.backend == "rasterio"]

,collection_concept_id,concept_id,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
10,C2021957295-LPCLOUD,G2705887278-LPCLOUD,s3://lp-prod-protected/HLSS30.020/HLS.S30.T33K...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[NIR_Narrow]
13,C2021957657-LPCLOUD,G2237588006-LPCLOUD,s3://lp-prod-protected/HLSL30.020/HLS.L30.T31V...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
115,C2076106409-LPCLOUD,G3552777394-LPCLOUD,s3://lp-prod-protected/ECO_L3T_JET.002/ECOv002...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
131,C3306855744-LPCLOUD,G3548868262-LPCLOUD,s3://lp-prod-protected/AST14DEM.004/AST14DEM_0...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
182,C2076104650-LPCLOUD,G3557453687-LPCLOUD,s3://lp-prod-protected/ECO_L4T_ESI.002/ECOv002...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
249,C2076102081-LPCLOUD,G3575616875-LPCLOUD,s3://lp-prod-protected/ECO_L4T_WUE.002/ECOv002...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
341,C2074860916-LPCLOUD,G3568654048-LPCLOUD,s3://lp-prod-protected/ECO_L3T_SM.002/ECOv002_...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]
351,C2519119034-LPCLOUD,G3515104466-LPCLOUD,s3://lp-prod-protected/OPERA_L3_DIST-ANN-HLS_V...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[Vegetation_disturbance_status]
450,C3256023791-NSIDC_CPRD,G3256043485-NSIDC_CPRD,s3://nsidc-cumulus-prod-protected/ASO/ASO_50M_...,rasterio,None,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[ASO SUPER Snow Water Equivalent [m]]
462,C2074877891-LPCLOUD,G3558289235-LPCLOUD,s3://lp-prod-protected/ECO_L3T_MET.002/ECOv002...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]


In [11]:
!aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
    s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
